# Run a PESTPP-IES experiment

In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pyemu
import matplotlib.pyplot as plt
import helpers

In [ ]:
_ = helpers.get_domain_map()

In [ ]:
# the existing dataset
org_working_d = "model_and_pest_files"
# the new dataset for this experiment
working_d = "model_and_pest_file_exp1"
# the master ies dir that will be created
master_d = "master_ies"
# number of parallel workers
num_workers = 10
# iteration number to visualize: if `None`, the final iteration is used
viz_noptmax = None

In [ ]:
assert os.path.exists(org_working_d)
if os.path.exists(working_d):
    shutil.rmtree(working_d)
shutil.copytree(org_working_d, working_d)

load the control file and check that it has had weights set

In [ ]:
pst = pyemu.Pst(os.path.join(working_d, "pest.pst"))
assert pst.nobs > pst.nnz_obs
assert pst.nnz_obs > 0

Modify any ies settings.  Lets use 4 iterations, 50 realizations, and a bad-phi-sigma of 1.75

In [ ]:
pst.control_data.noptmax = 4
pst.pestpp_options["ies_num_reals"] = 100
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
pst.pestpp_options["panther_agent_freeze_on_fail"] = True
# pst.pestpp_options.pop("ies_phi_factor_file",None)
# pst.pestpp_options["ies_n_iter_reinflate"] = [2, 999]
# pst.pestpp_options["ies_reinflate_num_reals"] = [50, -50]
# pst.pestpp_options["ies_multimodal_alpha"] = 0.99
# pst.pestpp_options.pop("ies_obs_en",None)
# pst.pestpp_options["ies_no_noise"] = True

pst.pestpp_options

Save the control file

In [ ]:
pst.write(os.path.join(working_d, "pest.pst"), version=2)

run PESTPP-IES

In [ ]:
if os.path.exists(master_d):
    shutil.rmtree(master_d)
os.makedirs(master_d)
pyemu.os_utils.start_workers(
    working_d,
    "pestpp-ies",
    "pest.pst",
    num_workers=num_workers,
    worker_root=master_d,
    master_dir=master_d,
)

Process and plot the results.  

Call `helpers.plot_ies_timeseries()`

In [ ]:
helpers.plot_ies_timeseries(master_d, noptmax=viz_noptmax)

Plot some property realizations using `helpers.plot_ies_properties()` by looping over each of these property tags:


In [ ]:
tags = [
    "npf-k-layer1",
    "npf-k-layer5",
    "npf-k33-layer3",
    "sto-ss-layer1",
    "sto-sy-layer1",
]

In [ ]:
for tag in tags:
    helpers.plot_ies_properties(master_d, tag, noptmax=viz_noptmax)

Plot the prior and posterior forecast histograms (the money plots!)

In [ ]:
helpers.plot_ies_forecasts(master_d, noptmax=viz_noptmax)

Explore other ies results via the `Pst.ies` handler:

In [ ]:
pst = pyemu.Pst(os.path.join(master_d, "pest.pst"))

Plot the `actual` and `meas`ured phi sequence across iterations

In [ ]:
pm, pa = pst.ies.phiactual, pst.ies.phimeas

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 10))
vals = np.log10(pm.iloc[:, 6:].values)
ivals = pm.iteration.values
_ = [axes[0].plot(ivals, vals[:, i], "0.5") for i in range(vals.shape[1])]
vals = np.log10(pa.iloc[:, 6:].values)
ivals = pa.iteration.values
_ = [axes[1].plot(ivals, vals[:, i], "0.5") for i in range(vals.shape[1])]

Visualize the parameter change summary across iterations

In [ ]:
for i in pst.ies.pcs.index.get_level_values(0).unique():
    print(i)
    lbnd = pst.ies.pcs.loc[i, "percent_at_near_lbound"]
    ubnd = pst.ies.pcs.loc[i, "percent_at_near_ubound"]
    fig, axes = plt.subplots(2, 1, figsize=(6, 6))
    lbnd.sort_values(inplace=False, ascending=False).iloc[:10].plot(
        kind="bar", ax=axes[0]
    )
    ubnd.sort_values(inplace=False, ascending=False).iloc[:10].plot(
        kind="bar", ax=axes[1]
    )
    axes[0].set_title("iteration {0} lower bound".format(i))
    axes[1].set_title("iteration {0} upper bound".format(i))
    plt.tight_layout()